In [1]:
'''1. Remove foreign records to a 'foreign dataframe' and delete them from this df. - applicable for  address
2.Find whether all phone numbers are empty, if empty drop those rows (8708)
3.Column Phone matches the Phone number in preferred phone number
Transfer records with non-null values in Phone to a new dataframe with columns EID and phone
Find the count of records (124000)
4. Null values in Phone: (36000 records) Create a new df:Check the content in preferred phone 
and grasp the preferred phone and add it to df with columns eid and phone

'''

import sys
import pandas as pd
from fuzzywuzzy import fuzz

df_txexAddr = pd.read_csv('TxExAddr.csv')
print("original count ",len(df_txexAddr))

df_Addr = df_txexAddr.dropna(how='all', subset=['Street'])
print("Count records after dropping records with no address : ",len(df_Addr))
#print(df_Addr.head(3))

df_Addr = df_Addr[['EID', 'Street']]
df_Addr['EID'] = df_Addr['EID'].str.upper()
print(len(df_Addr))
print(df_Addr.head(3))

original count  166745
Count records after dropping records with no address :  158723
158723
        EID                   Street
0    EVR253  1715 gosnell rd apt 104
1     CEN95        1808 Intervail Dr
2  GOLDENR1   5735 Diamond Point Cir


C:\Users\le5752\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# dividing the large dataframe into multiple small dataframes
df_addr_csv = df_Addr[100000:]
print(len(df_addr_csv))
print(df_addr_csv.head(3))
print(df_addr_csv.tail(3))

58723
             EID                  Street
104696    CMH573     541 Magazine Avenue
104697  JBAG7283  2800 Kirby Dr Apt A732
104698   SMS4275      14242 Bateau Drive
            EID                    Street
166742  YC28968  27403 Rosewood Valley Dr
166743  YC29226         6816 Schambray St
166744  YC29237       16852 Tableland Trl


In [3]:
raw_eid = df_addr_csv['EID'].dropna()
eid = raw_eid.tolist()
#print(len(eid))
# How many elements each list should have 
n = 1000
pid = {}
for i in range((len(eid) + n - 1) // n ):
    #globals()["eid" + str(i)] = eid[i * n:(i + 1) * n]
    pid[i] = eid[i * n:(i + 1) * n]

In [4]:
import cx_Oracle
connection = cx_Oracle.connect("vipapp/ZJf3__6qjRt8@exa1-scan.austin.utexas.edu:1525/viprp_pdb.austin.utexas.edu")
cursor = connection.cursor()
df_addr_db = pd.DataFrame(columns = ['EID','Address1','Address2','City','State','Zip Code'])
for i in range(50):
    cursor.execute("select UT_EID, vi_addr_line1, vi_addr_line2, vi_city, vi_state,vi_zip from vi_vips_main m, vi_vips_addresses a  where m.ALUMNI_ID = a.ALUMNI_ID AND m.ut_eid IN %s" % str(tuple(pid[i])).replace(',)',')'))
    results = cursor.fetchall()
    df1 = pd.DataFrame(results, columns = ['EID','Address1','Address2','City','State','Zip Code'])
    df_addr_db = df_addr_db.append(df1,ignore_index=True)
cursor.close()
connection.close()                   

In [5]:
print(len(df_addr_db))
df_addr_db = df_addr_db.drop_duplicates()
print(len(df_addr_db))
print(df_addr_db.head(3))

60226
60162
        EID                  Address1 Address2     City State     Zip Code
0  0227QQEI         1687 Rachel Place     None    Vidor    TX  77662670687
1  0227YDYA     7819 Leaf Point Court     None  Houston    TX  77095417919
2  0293ZGHC  22126 Conquistador Drive     None   Crosby    TX  77532313326


In [6]:
addr_diff = pd.DataFrame()
for i in range(len(df_addr_csv)):
#for i in (11,44,55,66,123,133,143,333,445,667,246,988):
    flag = 0
    id_csv = df_addr_csv.iloc[i,0]
    csv_addr = df_addr_csv.iloc[i,1]
    cmp_dbaddr = df_addr_db.loc[df_addr_db['EID'] == id_csv]
    #print(cmp_dbaddr)
    #print(df_addr_csv.loc[df_addr_csv['EID'] == id_csv])
    #print("---")
    
    for j in range(len(cmp_dbaddr)):
        db_addr = cmp_dbaddr.iloc[j,1]
        Token_Set_Ratio = fuzz.token_set_ratio(csv_addr,db_addr)
        if (Token_Set_Ratio > 77):
            flag = 0
            break
        else:
            flag = 1
    
    #print("flag = ", flag)
    
    if (flag == 1):
        addrlist = []
        for k in range(len(cmp_dbaddr)):
            temp = (cmp_dbaddr.loc[cmp_dbaddr['EID'] == id_csv]).iloc[:,1]
            addrlist = temp.values.tolist()
        addr_diff = addr_diff.append({'EID':id_csv, 'DB_Address':addrlist,'CSV_Address':csv_addr}, ignore_index=True)
#print("----------------------------------------------------------------")
#print(addr_diff)
#print("----------------------------------------------------------------")

In [7]:
print(addr_diff.head(11))
print(len(addr_diff))

                               CSV_Address  \
0                   2800 Kirby Dr Apt A732   
1   141 Old Short Hills Road Apartment 114   
2                               PO Box 654   
3                    3000 Rivercrest Drive   
4                     6318 Box Bluff Court   
5                           1501 Daly Road   
6                     2408 Leon St Apt 115   
7                     1507 Pease Rd Apt #5   
8                        6513 Cahoba Drive   
9                           P. O. Box 7437   
10                   2515 Pearl St Apt 504   

                                     DB_Address       EID  
0         [1915 Country Club, 2800 Kirby Drive]  JBAG7283  
1    [1606 Preston Trail, 8616 Beetle Nut Lane]  COTTONCE  
2                            [106 Shiloh Drive]       MRB  
3                        [5205 Leralynn Street]    MFR297  
4                       [10811 Croftmore Drive]  WOODSSA3  
5   [2401 El Salido Pkwy, 24127 Prestige Drive]   MJD2785  
6        [710 Countryside D

In [8]:
addr_diff.to_csv(r'C:\Users\le5752\Documents\DB_Python\addr3.csv', index = False)